# Projeto de implementação 1
### Problema do Caixeiro Viajante
Alunos: Leticia Bossatto Marchezi (791003) e Luís Augusto Simas do Nascimento (790828)


Disciplina: Inteligência Artificial


Implementação de uma solução para o problema do Caixeiro Viajante usando algoritmos evolutivos.

Importando bibliotecas

In [ ]:
import numpy as np
import random

## Variáveis globais

In [ ]:
qtde_cidades = 6
qtde_populacao = 20
max_geracoes = 50
qtde_elite = 2

## Gerando as distâncias

Gerando a matriz com distâncias aleatórias entre as cidades.
Os índices x e y representam as cidades de origem e destino, sendo a matriz espelhada em relação a diagonal principal.


In [ ]:
distancia = np.zeros((qtde_cidades,qtde_cidades), dtype=int)

In [ ]:
for x in range(qtde_cidades):
  for y in range(x+1,qtde_cidades):
      distancia[x][y] = random.randint(0,100)
      distancia[y][x] = distancia[x][y]

In [ ]:
distancia

array([[ 0, 99, 85, 65, 14, 36],
       [99,  0, 80, 42, 49, 98],
       [85, 80,  0, 62, 68, 30],
       [65, 42, 62,  0, 86,  8],
       [14, 49, 68, 86,  0, 43],
       [36, 98, 30,  8, 43,  0]])

## Representação dos indivíduos

Os indivíduos são representados por um vetor de inteiros contendo as cidades visitadas em ordem. Note que a cidade 0 é omitida pois ela é, por convenção, sempre a cidade inicial e final. Além disso, a cidade 0 não é considerada como um dos valores possíveis do vetor, variando apenas entre o intervalo fechado $[1,5]$.

Essa decisão foi tomada com base no fato de que, por definição, a primeira e a última cidade são sempre a cidade $0$, portanto omiti-las permite simplificar a representação dos indivíduos e, consequentemente, os operadores que atuam sobre estes.

Por exemplo, uma solução possível para o problema é o caminho `0-1-3-2-4-5-0`, que é representado pelo vetor `[1, 3, 2, 4, 5]`

## Implementação dos indivíduos da população inicial

Os indivíduos são construídos usando arrays provenientes da biblioteca `numpy`. Um indivíduo é gerado pela função `randint` da biblioteca `random`, que sorteia 5 números aleatórios no intervalo $[1,5]$. 

Além disso, uma geração é um array de dimensão `[m][n-1]`, sendo `m` a quantidade de indivíduos da geração e `n` a quantidade de cidades. As gerações são declaradas com elementos nulos e cada indivíduo é atribuido em um laço de repetição.

In [ ]:
populacao = np.zeros((qtde_populacao, qtde_cidades - 1), dtype=int)
for x in range(qtde_populacao):
  for y in range(qtde_cidades - 1):
    populacao[x][y] = random.randint(1, qtde_cidades - 1)

In [ ]:
populacao

array([[2, 2, 2, 4, 4],
       [1, 1, 4, 2, 2],
       [1, 5, 2, 1, 3],
       [3, 3, 3, 5, 5],
       [2, 5, 1, 2, 1],
       [5, 2, 2, 5, 3],
       [2, 4, 2, 2, 2],
       [5, 1, 2, 5, 4],
       [5, 1, 2, 4, 5],
       [3, 1, 3, 4, 3],
       [2, 4, 5, 1, 2],
       [5, 5, 5, 5, 1],
       [2, 4, 2, 3, 3],
       [5, 3, 5, 4, 3],
       [3, 4, 4, 1, 3],
       [1, 4, 1, 1, 3],
       [1, 5, 3, 1, 4],
       [3, 5, 3, 4, 5],
       [5, 4, 4, 2, 5],
       [2, 4, 5, 4, 5]])

## Solução ótima

Implementou-se também um algoritmo para obter a solução ótima com o objetivo de testar o algoritmo evolutivo. Esse algoritmo se baseia em um método de força bruta, gerando todas as permutações de caminhos possíveis (sem repetição de cidades) e minimizando a função de *fitness*, dessa forma encontrando a solução ótima (ou uma delas).

Vale notar que, como não há restrições entre as distâncias entre as cidades ou a ordem do caminho, é possível que exista mais de uma solução ótima.

In [ ]:
import itertools

def solucao_otima():
  permutations = list(itertools.permutations(range(1, qtde_cidades)))

  fitnesses = list(map(fitness, permutations))
  fit = min(fitnesses)
  ind = permutations[fitnesses.index(fit)]

  return {
      "fitness": fit,
      "individuo": ind
  }

## Fitness

A função de *fitness* calcula a distância total do caminho com base na matriz de distâncias. A distância total é dada pela soma simples das distâncias entre cada viagem do caminho. Note que como os indivíduos são representados omitindo a cidade inicial e final (pois são sempre a cidade 0), é necessário uma lógica adicional para calcular a distância da primeira e última viagem.

Note também que no processo de obtenção da população inicial não são realizadas nenhuma verificação de que os indivíduos representam um caminho que passa por todas as cidades. Dessa forma, é necessário implementar alguma forma de penalizar os indivíduos que infringem as regras estabelecidas na definição do problema. Essa penalização foi implementada na própria função de fitness, através da adição do produto do número de cidades que não foram percorridas por um fator arbitrário `e`.

Veja ainda que como o problema trata de minimizar a distância do trajeto, quanto **menor** a função de fitness, **mais apto** é o indivíduo, e esse fato é assumido ao longo do algoritmo, principalmente na implementação dos operadores evolutivos.

In [ ]:
def fitness(ind):
  f = 0 # fitness
  c = 0 # cidades faltando
  e = 1000 # penalização

  for i in range(qtde_cidades):
    anterior = 0 if i == 0 else ind[i - 1]
    atual = 0 if i == qtde_cidades - 1 else ind[i]
    f += distancia[anterior][atual]

  c = qtde_cidades - 1 - len(np.unique(ind))
  
  return f + c * e

def fitness_populacao(populacao):
  return list(map(fitness, populacao))

## Seleção proporcional

A forma de seleção de indivíduos para o cruzamento ocorre de acordo com o valor calculado pela função *fitness*. Como explicitado, quanto menor o valor de fitness do indivíduo, mais apto ele é, logo a probabilidade de um indivíduo ser sorteado deve ser proporcional ao inverso do seu valor de fitness.

A escolha é feita a partir do método `choices` da biblioteca `random`, que permite o sorteio de elementos com pesos, definindo a probabilidade do indivíduo entre a população inteira.

Por fim, a função garante que dois elementos diferentes sejam escolhidos.


In [ ]:
def selecao_proporcional(populacao):
  fit = fitness_populacao(populacao)
  fit_inverso = list(map(lambda x: 1 / x, fit))
  ind1 = 0
  ind2 = 0
  
  while np.array_equal(ind1, ind2):
    ind1, ind2 = random.choices(populacao, weights = fit_inverso, k = 2)

  return ind1, ind2

## Cruzamento
Cruza os elementos de 2 indivíduos até um ponto aleatório, exceto extremidades

In [ ]:
def cruzamento(ind1, ind2):
  ponto_cruzamento = random.randint(1, qtde_cidades - 2)
  
  filho1 = np.concatenate([ind1[:ponto_cruzamento], ind2[ponto_cruzamento:]])
  filho2 = np.concatenate([ind2[:ponto_cruzamento], ind1[ponto_cruzamento:]])

  return filho1, filho2


## Mutação


A mutação dos indivíduos se dá pela troca de posição entre elementos de um genoma. Esse método foi escolhido pois provoca a alteração de trajeto, já que a posição das cidades é relevante para o problema. Assim, essa ação interefere diretamente no valor da função *fitness* e pode gerar caminhos mais otimizados ou não.

Porém, o sistema de mutação não implica na adição de uma cidade nova no trajeto, sendo o cruzamento o responsável por tal efeito.

In [ ]:
def mutacao(ind, taxa = 0.5):
  if random.choices([True, False], weights = [taxa, 1 - taxa]):
    i = random.randint(0, len(ind) - 1)
    j = random.randint(0, len(ind) - 1)
  
    ind[i], ind[j] = ind[j], ind[i]

  return ind


## Elitismo

O objetivo do elitismo é manter uma seleção dos $n$ melhores indivíduos para a próxima geração. A implementação da função de elitismo nesse caso é simples, sendo possível especificar através de um parâmetros o número de indivíduos a mantido da população.

In [ ]:
def elitismo(populacao, n):
  elite = sorted(populacao, key = fitness)

  return elite[0:n]

# Programa Principal

O programa executa um laço de repetição enquanto o número de gerações não alcançar o limite estabelecido ou até que uma das soluções ótimas seja gerada.

A cada iteração, os `n` elementos com melhor *fitness* são selecionados por elitismo e armazenados na nova população. Após isso, ocorre o cruzamento entre os indivíduos e a mutação entre os filhos, gerando a população da próxima iteração.



In [ ]:
import functools
otimo = solucao_otima()
i = 0

# Helper para exibir as estatísticas de cada geração
def print_ind(ind, n):
  caminho = np.concatenate([[0], ind, [0]])
  fit = fitness(ind)
  fit_pop = fitness_populacao(populacao)
  media_fit = functools.reduce(lambda a, x: a + x, fit_pop) / qtde_populacao

  print("{:02d} ".format(n), " ", caminho, " ", fit," ", media_fit)
  
print("gen   melhor            fit   média população ")

while i < max_geracoes and fitness(elitismo(populacao, 1)[0]) != otimo["fitness"]:
  populacao_nova = np.zeros((qtde_populacao,qtde_cidades - 1), dtype=int)

  fit = fitness_populacao(populacao)
  
  elite = elitismo(populacao, qtde_elite)
  print_ind(elite[0], i)
  populacao_nova[0:qtde_elite] = elite
  
  for num_filhos in range(qtde_elite,qtde_populacao, 2):
    pai1, pai2 = selecao_proporcional(populacao)
    filhos = cruzamento(pai1, pai2)
    mutados = list(map(mutacao, filhos))
    populacao_nova[num_filhos:num_filhos + 2] = mutados

  populacao = populacao_nova.copy()
  i = i + 1

elite = elitismo(populacao, qtde_elite)

print_ind(elite[0], i)

print()
print("solução ótima:      ", np.concatenate([[0], otimo["individuo"], [0]]), " ", otimo["fitness"])
print("solução encontrada: ", np.concatenate([[0], elite[0], [0]]), " ", fitness(elite[0]))

gen   melhor            fit   média população 
00    [0 4 1 3 5 2 0]   228   1406.1

solução ótima:       [0 2 5 3 1 4 0]   228
solução encontrada:  [0 4 1 3 5 2 0]   228
